In [310]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date, datetime, time, timedelta
import re
import statistics
from classes import Speed, Direction
from Time import Hours, Date, DateTime

In [320]:
class DataFrame(pd.DataFrame):
    def __init__(self, df):
        processed_df = self.process_df(df)
        super().__init__(df)

In [322]:
class PriceDemand(DataFrame):
    
    @staticmethod
    def process_df(price_demand):
        year_dates = DateTime(price_demand["SETTLEMENTDATE"]).convert_to_year()
        price_demand["Date"] = year_dates
        return price_demand
   
    def get_grouped(price_demand):
        agg_params = {"TOTALDEMAND": "sum", "PRICESURGE": lambda surge_series: surge_series.loc[surge_series].size}
        grouped_price_demand = price_demand.groupby(["REGION", "Date"]).agg(agg_params)
        return grouped_price_demand

In [312]:
class City(pd.DataFrame):
    
    def __init__(self, city_df, state_code):
        city_df["REGION"] = state_code;
        city_df["Date"] = Date(city_df["Date"]).time_objects
        city_df.set_index(["Date", "REGION"], inplace=True)
        super().__init__(city_df)

In [325]:
class Cities(pd.DataFrame):
    
    @staticmethod
    def get_all_cities():
        WEATHER_DIR = "./weather/"
        CITY_SUFFIXES = ["_melbourne", "_sydney", "_adelaide", "_brisbane"]
        STATE_CODES = ["VIC1", "NSW1", "SA1", "QLD1"]
        CITY_DIRS = [WEATHER_DIR + "weather" + suffix + ".csv" for suffix in CITY_SUFFIXES]
        cities = []
        for i in range(len(CITY_DIRS)):
            #[(df, statecode), (df, statecode)]
            cities.append((pd.read_csv(CITY_DIRS[i]), STATE_CODES[i]))    
        return cities

    
    def __init__(self):
        cities_dataframes = self.get_all_cities()
        City_array = [City(*city) for city in cities_dataframes]
        Cities_dataframe = pd.concat(City_array, axis=0).reset_index()
        super().__init__(Cities_dataframe)

    def price_join(self, grouped_price_demand):
        return self.join(grouped_price_demand, on=["REGION", "Date"], how='right')

In [326]:
cities = Cities()

In [327]:
price_demand_grouped = PriceDemand(price_demand).get_grouped()

In [328]:
cities.price_join(price_demand_grouped)

,Date,REGION,Minimum temperature (°C),Maximum temperature (°C),Rainfall (mm),Evaporation (mm),Sunshine (hours),Direction of maximum wind gust,Speed of maximum wind gust (km/h),Time of maximum wind gust,...,9am wind speed (km/h),9am MSL pressure (hPa),3pm Temperature (°C),3pm relative humidity (%),3pm cloud amount (oktas),3pm wind direction,3pm wind speed (km/h),3pm MSL pressure (hPa),TOTALDEMAND,PRICESURGE
424,2021-02-01,NSW1,20.7,28.9,0.8,0.4,6.8,NNE,44.0,17:26,...,7,1016.1,28.3,68.0,1.0,ESE,17,1012.3,368948.08,0
425,2021-02-02,NSW1,17.5,25.8,17.4,7.6,5.9,SSW,43.0,04:49,...,17,1010.5,23.2,60.0,2.0,SE,24,1010.5,355162.09,0
426,2021-02-03,NSW1,17.5,25.8,5.0,7.8,9.3,S,31.0,09:10,...,15,1013.7,24.7,56.0,5.0,ESE,17,1012.6,347299.42,0
427,2021-02-04,NSW1,17.9,28.9,0.0,6.8,10.1,NE,48.0,13:58,...,Calm,1013.4,27.7,62.0,5.0,NE,30,1010.0,375599.02,0
428,2021-02-05,NSW1,20.8,28.3,0.0,8.4,11.2,NE,54.0,12:25,...,17,1012.1,28.1,71.0,2.0,NE,33,1009.0,393493.57,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,2022-03-14,VIC1,16.4,28.0,0.4,5.0,9.3,SSE,31.0,15:58,...,Calm,1021.2,25.6,64.0,6.0,SW,9,1018.7,1258619.68,0
407,2022-03-15,VIC1,18.3,27.5,2.0,6.2,3.6,NNW,31.0,10:16,...,7,1019.8,24.8,57.0,7.0,SSW,13,1018.2,1441696.85,0
408,2022-03-16,VIC1,18.9,23.2,0.0,3.6,1.7,NNE,41.0,10:29,...,9,1018.8,22.2,58.0,7.0,N,19,1016.4,1391761.70,0
409,2022-03-17,VIC1,17.7,29.2,0.0,4.6,5.0,N,33.0,09:28,...,6,1015.2,27.8,53.0,5.0,NNW,11,1013.0,1469162.52,0
